# Building R Packages Corpus for Question Answeing System

In [ ]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

In [ ]:
# Reading R qas dataset from the file
import pickle

# Read list to memory
def read_r_qas_dataset():
    with open(r'/content/drive/MyDrive/Colab Notebooks/data/r_qas_dataset_qa1', 'rb') as fp:
        return pickle.load(fp)

r_qas_dataset = read_r_qas_dataset()

In [ ]:
import json
# print(len(r_qas_dataset))
print(json.dumps(r_qas_dataset[:10], sort_keys=True, indent=4))

In [ ]:
! pip install simpletransformers
! pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# for item in r_qas_dataset:
#   # print(item['qas'])
#   for qas in item['qas']:
#     # print(qas['answers'])
#     for qa in qas['answers']:
#       qa['answer_start'] += 1


In [ ]:
# for item in r_qas_dataset[:10]:
#   print(item['qas'])

In [ ]:
import json
with open(r"/content/drive/MyDrive/Colab Notebooks/data/r_qas_dataset_train.json", "r") as read_file:
    train = json.load(read_file)

In [ ]:
train = r_qas_dataset[:10]
test = r_qas_dataset[10:]

In [ ]:
# import numpy as np
# from sklearn.model_selection import train_test_split

# train, test = train_test_split(r_qas_dataset[:10], test_size=0.33, random_state=42)

In [ ]:
print(len(train))
print(len(test))

10
5


In [ ]:
train[20:]

In [ ]:
import re
dt_context = "ABCanalysis: Computed ABC Analysis For a given data set, the package provides a novel method of computing precise limits to acquire subsets which are easily interpreted. Closely related to the Lorenz curve, the ABC curve visualizes the data by graphically representing the cumulative distribution function. Based on an ABC analysis the algorithm calculates, with the help of the ABC curve, the optimal limits by exploiting the mathematical properties pertaining to distribution of analyzed items. The data containing positive values is divided into three disjoint subsets A, B and C, with subset A comprising very profitable values, i.e. largest data values (the important few), subset B comprising values where the yield equals to the effort required to obtain it, and the subset C comprising of non-profitable values, i.e., the smallest data sets (the trivial many). Package is based on Computed ABC Analysis for rational Selection of most informative Variables in multivariate Data, PLoS One. Ultsch. A., Lotsch J. (2015) <doi:10.1371/journal.pone.0129767>."
dt_answer = 'abcADM'
dt_answer_start = re.search(r'\b({})\b'.format(dt_answer.split(' ', 1)[0]), dt_context)
print(dt_answer_start.start())

AttributeError: ignored

In [ ]:
test[:10]

In [ ]:
# train = r_qas_dataset
train

[{'context': 'A system for declaratively creating graphics, based on The Grammar of Graphics. ggplot2 is used to Create Elegant Data Visualisations Using the Grammar of Graphics. You provide the data, tell ggplot2 how to map variables to aesthetics, what graphical primitives to use, and it takes care of the details.',
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'What is the R package for data visualization?',
    'answers': [{'text': 'ggplot2 is used to Create Elegant Data Visualisations Using the Grammar of Graphics',
      'answer_start': 81}]},
   {'id': '00002',
    'is_impossible': False,
    'question': 'What is the R package for grammar of graphics?',
    'answers': [{'text': 'ggplot2 is used to Create Elegant Data Visualisations Using the Grammar of Graphics',
      'answer_start': 81}]}]},
 {'context': ' Shiny is an R package that makes it easy to build interactive web apps straight from R. You can host standalone apps on a webpage or embed them in R 

In [ ]:
with open(r"/content/drive/MyDrive/Colab Notebooks/data/r_qas_dataset_test.json", "r") as read_file:
    test = json.load(read_file)

In [ ]:
test

In [ ]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [ ]:
model_type="bert"
model_name= "bert-base-cased"

# model_type == "distilbert"
# model_name = "distilbert-base-cased"


# if model_type == "bert":
#     model_name = "bert-base-cased"

# elif model_type == "roberta":
#     model_name = "roberta-base"

# elif model_type == "distilbert":
#     model_name = "distilbert-base-cased"

# elif model_type == "distilroberta":
#     model_type = "roberta"
#     model_name = "distilroberta-base"

# elif model_type == "electra-base":
#     model_type = "electra"
#     model_name = "google/electra-base-discriminator"

# elif model_type == "electra-small":
#     model_type = "electra"
#     model_name = "google/electra-small-discriminator"

# elif model_type == "xlnet":
#     model_name = "xlnet-base-cased"

In [ ]:
# Configure the model 
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size=1
model_args.num_train_epochs=5

In [ ]:
### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":1,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [ ]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args, use_cuda=True
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

In [ ]:
### Remove output folder
!rm -rf outputs

In [ ]:
# Train the model
model.train_model(train, eval_data=test, verbose=True,)
# model.train_model(train)

add example index and unique id: 100%|██████████| 5/5 [00:00<00:00, 38060.83it/s]


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

correct,▁▁▁▁▁
eval_loss,█▇▅▃▁
global_step,▁▃▅▆█
incorrect,█▁▁▁▁
similar,▁████
train_loss,██▄▂▁
correct,1
eval_loss,-1.08203
global_step,5
incorrect,0
similar,2


Running Epoch 0 of 5:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


...

Running Epoch 3 of 5:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

(5,
 {'global_step': [1, 2, 3, 4, 5],
  'correct': [0, 1, 1, 1, 1],
  'similar': [1, 2, 2, 2, 2],
  'incorrect': [2, 0, 0, 0, 0],
  'train_loss': [4.984375,
   5.005468845367432,
   4.41796875,
   3.994140625,
   3.7398438453674316],
  'eval_loss': [-0.1060791015625,
   -0.12939453125,
   -0.1651611328125,
   -0.1962890625,
   -0.216552734375]})

In [ ]:
# Evaluate the model
result, texts = model.eval_model(test)

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Make predictions with the model
to_predict = [
    {
        "context": "gtrendsR is the R package to perform and display Google Trends queries. An interface for retrieving and displaying the information returned online by Google Trends is provided. Trends (number of hits) over the time as well as geographic representation of the results can be displayed.",
        "qas": [
            {
                "question": "what is package for google trends?",
                "id": "0",
            }
        ],
    }
]


In [ ]:
answers, probabilities = model.predict(to_predict)

print(answers)

print(probabilities)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2002.05it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['gtrendsR is the R package to perform and display Google Trends queries. An interface for retrieving and displaying the information returned online by Google Trends is provided.']}]
[{'id': '0', 'probability': [0.8082186275507353]}]
